What are you looking to get out of this data? 

Part A
- My method of using reviews good? Assuming that if not review (and there are atleast 10) that no used tires sold.
- No info boolean

Part B
- Do you want map iamge or browsable map? Will you just be loading this into ArcGIS/ Google Earth? 
- Both leaflet and kml

- What demographic are used tire shops serving ?

- Corrolation matrix 

Part C
- By pop density do you mean number of people per store from sample or number of people per all retail stores? Just get gross and pop density 
- Digital divide index may be hard to get quikly, will need to requet it from a curosry search. May be able to use ACS for this tho USE CENSUS 
- Other variables?
- By shapefiles I assume this is the census tract (or whatever geo area each variable corresponds to) not the stores footprint

Part D
- Histograms as image or interactive charts?
- Y asix num shops
- X axis vars from part C

- Do leaflet.js and images 

Part E
- Multivariate map is difficult to interpret when >3 variables. I would suggest a map viz with variable selectors and a table for looking 
at all together. Also map viz using census tracts will be vary to use when zoomed out and stores are rather sparse. 

- Research google places api for tires stores 

- Wrtie beifef analysis of finds 

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import re
from tqdm import tqdm

import config as cfg

executable_path = {'executable_path': 'chromedriver.exe'}
pd.set_option('display.max_rows', 500)

In [3]:
url = 'https://www.catiredealers.com/Members/Locations'
# headers={
#     "authority": "www.catiredealers.com",
#     "method": "GET",
#     "path": "/Members/Locations",
#     "scheme": "http",
#     "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
#     "accept-encoding": "gzip, deflate, br",
#     "accept-language": "en-US,en;q=0.9,ja;q=0.8,pt;q=0.7",
#     "cache-control": "max-age=0",
#     "cookie": "__cfduid=d4eb66f9debe2ed5126406468f7eb460b1603798162; .ASPXANONYMOUS=zR64xYie0t82KtbYoh3xd4VmwKrdWdvzCeNbx8pcTZGYLaLUizRtSpFfXYXUryVV_N3d6sj_hPaKBy7mHhGiLM0G0nimI6nwVJ9GRAOqH4yUL3qY0; locDistances=; _ga=GA1.2.55266223.1603798175; _gid=GA1.2.1619571755.1603798175; dnn_IsMobile=False; language=en-US; BIGipServernetdriven_SSL_POOL=545722634.47873.0000; _pk_id.967be4f9-5542-48c3-a90b-2b5b7e580f8d.7a3f=ffeb111edeb361ce.1603798176.3.1603893382.1603893382.; _pk_ses.967be4f9-5542-48c3-a90b-2b5b7e580f8d.7a3f=*",
#     "sec-fetch-dest": "document",
#     "sec-fetch-mode": "navigate",
#     "sec-fetch-site": "none",
#     "Sec-Fetch-User": "?1",
#     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36",
# }
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()
locations = soup.find("div", id="LocationListView")

In [69]:
df = pd.DataFrame(columns=["name", "address", "phone_number"])

for location in locations.find_all("div", id=re.compile("loc(.*)")):
    info = location.find("div", class_="locationInfo")
    name = info.find("strong").text
    addr_1 = str(info).split("<br/>")[2].strip()
    addr_2 = str(info).split("<br/>")[3].strip().replace("</div>","")
    addr = addr_1+" "+addr_2
    addr = " ".join(addr.strip().replace("\n","").split())
    try:
        phone_number = "".join(filter(str.isdigit, location.find("div", class_="locphone").find("a").text))
    except Exception as e:
#         print(e)
        phone_number = None
    
    df = df.append({"name":name, "address": addr, "phone_number": phone_number}, ignore_index=True)
        

In [73]:
def get_meta_data(address):
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={cfg.GKEY}'
    target_url = target_url.replace("#","")
    geo_data = requests.get(target_url).json()

    lat = geo_data['results'][0]['geometry']['location']['lat']
    lng = geo_data['results'][0]['geometry']['location']['lng']

    address_list = []
    city = ''
    state = ''
    
    if geo_data['status']=='ZERO_RESULTS':
        return None
    
    for item in geo_data['results'][0]['address_components']:
        if 'street_number' in item['types']:
            address_list.append(item['long_name'])

        if 'route' in item['types']:
            address_list.append(item['long_name'])

        if 'locality' in item['types']:
            city = item['long_name']

        if 'administrative_area_level_1' in item['types']:
            state = item['long_name']

    street = ' '.join(address_list).strip()
    # print(street, city, state)

    benchmark = "Public_AR_ACS2019"
    vintage = "ACS2019_ACS2019"
    layers = 8

    url = f'https://geocoding.geo.census.gov/geocoder/geographies/address?street=\
                {street}&city={city}&state={state}&benchmark={benchmark}&vintage={vintage}&layers={layers}&format=json'
    response = requests.get(url)
    geo_id_dict = response.json()

    try:
        tract = geo_id_dict['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['TRACT']
        state = geo_id_dict['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['STATE']
        county = geo_id_dict['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['COUNTY']
        geo_id = state+county+tract
    except:
        geo_id = None
    
    return lat, lng, geo_id
    

In [77]:
df['lat'] = None
df['lng'] = None
df['geo_id'] = None

for n, row in tqdm(df.iterrows()):
    lat, lng, geo_id = get_meta_data(row.address)
    df.loc[n, "lat"] = lat
    df.loc[n, "lng"] = lng
    df.loc[n, "geo_id"] = geo_id

127it [04:30,  2.13s/it]


In [76]:
df.to_csv("locations.csv")

In [ ]:
for i in df.lng:
    print(i)

In [9]:
import geopandas as gp
from shapely.geometry import Point

In [70]:
df = pd.read_csv("Tire Shops - Locations.csv")
df = df.loc[df['Exclude']==False]
df.drop('Exclude', axis=1, inplace=True)
df['phone_number'] = df.phone_number.apply(lambda x: phonenumbers.format_number(phonenumbers.parse(str(x), 'US'), phonenumbers.PhoneNumberFormat.NATIONAL))
df.to_csv('tire_shops.csv', index=False)
df['Sells Used Tires?'] = df['Sells Used Tires?'].apply(lambda x: 1 if x else 0)
gdf = gp.GeoDataFrame(df, crs="EPSG:4326", geometry=[Point(x,y) for x,y in zip(df.lng,df.lat)])

In [71]:
import fiona
fiona.supported_drivers['KML'] = 'rw'
gdf.to_file('tire_shops.kml', driver='KML')

c:\users\art perkitny\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [72]:
gdf.to_file('tire_shops.shp')

c:\users\art perkitny\anaconda3\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
